# This location analysis aims to assist Confectionery Ltd in making a decision to choose a single location to open a new shop from given two lucrative locations.
### First we import the necessary libraries

In [4]:
from pandas.io.html import read_html
import os
import requests
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

# We download the data

In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    ))

The dataframe has 5 boroughs and 306 neighborhoods.


In [14]:
# Find out how many Boroughs are there in New York
neighborhoods["Borough"].value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

In [15]:
# Find the coordinates for New York
address = 'New York City, NY'

geolocator = Nominatim(user_agent="capstone")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [16]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [17]:
#Filter out Manhattan
man_data = neighborhoods[neighborhoods['Borough'] =='Manhattan'].reset_index(drop=True)
man_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [18]:
# Get a feel of the neighborhoods in Mnhattan
man_data["Neighborhood"].value_counts()

Manhattanville         1
Civic Center           1
Tribeca                1
Upper East Side        1
Little Italy           1
Gramercy               1
Carnegie Hill          1
Inwood                 1
Lincoln Square         1
Tudor City             1
West Village           1
Central Harlem         1
Murray Hill            1
Greenwich Village      1
Hamilton Heights       1
Yorkville              1
Midtown South          1
Lenox Hill             1
East Village           1
Morningside Heights    1
Financial District     1
Chelsea                1
Washington Heights     1
Sutton Place           1
Stuyvesant Town        1
Chinatown              1
Midtown                1
Lower East Side        1
Manhattan Valley       1
Battery Park City      1
Roosevelt Island       1
Turtle Bay             1
Upper West Side        1
Noho                   1
Hudson Yards           1
East Harlem            1
Clinton                1
Soho                   1
Marble Hill            1
Flatiron               1


In [19]:
# create map of Manhattan using latitude and longitude values
map_man = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(man_data['Latitude'], man_data['Longitude'], man_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_man)  
    
map_man

In [20]:
# Get the coordinates for Manhattan
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [42]:
# The code was removed by Watson Studio for sharing.

Your credntials are hidden!


In [22]:
# address for Location 1
address1 = "332 E 76th St, New York, NY"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address1)
latitude1 = location.latitude
longitude1 = location.longitude
print(latitude1, longitude1)

40.6304522244898 -74.0271703877551


In [23]:
# address for Location 2
address2 = "601 Lexington Ave, New York, NY"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address2)
latitude2 = location.latitude
longitude2 = location.longitude
print(latitude2, longitude2)

40.7583358 -73.969915265694


In [24]:
# search for schools in the area
search_query = 'School'
radius = 500
print(search_query + ' .... OK!')

School .... OK!


In [25]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=D0Y1ID3LKSEQNR01FLFZFRQR0AO45ISBTXZK3TMJKW3OXUUX&client_secret=ZDSX1VXNKJ1LOXXLPINFLJYOWMIWTN2AKERZQ50ZKAMR0TLP&ll=40.6304522244898,-74.0271703877551&v=20191230&query=School&radius=500&limit=30'

In [26]:
results1 = requests.get(url).json()
results1

{'meta': {'code': 200, 'requestId': '5e0aaf17b57e88001b49831a'},
 'response': {'venues': [{'id': '4b92c39ef964a520991934e3',
    'name': 'Vicky Simegiatos Dance School',
    'location': {'address': '7110 3rd Ave',
     'crossStreet': 'btwn 71st and 72nd St.',
     'lat': 40.63394249448927,
     'lng': -74.02680656605341,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.63394249448927,
       'lng': -74.02680656605341}],
     'distance': 389,
     'postalCode': '11209',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['7110 3rd Ave (btwn 71st and 72nd St.)',
      'Brooklyn, NY 11209',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d134941735',
      'name': 'Dance Studio',
      'pluralName': 'Dance Studios',
      'shortName': 'Dance Studio',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_dancestudio_',
       'suffix': '.png'},
  

In [27]:
# assign relevant part of JSON to schools1
schools1 = results1['response']['venues']

# tranform schools1 into a dataframe
schools1_df = json_normalize(schools1)
schools1_df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",False,4b92c39ef964a520991934e3,7110 3rd Ave,US,Brooklyn,United States,btwn 71st and 72nd St.,389,"[7110 3rd Ave (btwn 71st and 72nd St.), Brookl...","[{'label': 'display', 'lat': 40.63394249448927...",40.633942,-74.026807,11209,NY,Vicky Simegiatos Dance School,v-1577758462
1,"[{'id': '4bf58dd8d48988d1a8941735', 'name': 'G...",False,4b2791e3f964a5207b8724e3,211 72nd St,US,Brooklyn,United States,Ridge Avenue,430,"[211 72nd St (Ridge Avenue), Brooklyn, NY 1120...","[{'label': 'display', 'lat': 40.63411946924494...",40.634119,-74.028804,11209,NY,PS 102 - The Bay View School,v-1577758462
2,"[{'id': '4f4533804b9074f6e4fb0105', 'name': 'E...",False,4c8a53481eafb1f7d9207935,440 Ovington Ave,US,Brooklyn,United States,NaN,467,"[440 Ovington Ave, Brooklyn, NY 11209, United ...","[{'label': 'display', 'lat': 40.63303383059814...",40.633034,-74.022808,11209,NY,Lutheran elementary school,v-1577758462
3,"[{'id': '4f4533814b9074f6e4fb0106', 'name': 'M...",False,505c6db0e4b08887c5814fcb,NaN,US,Brooklyn,United States,NaN,522,"[Brooklyn, NY 11209, United States]","[{'label': 'display', 'lat': 40.62755320651353...",40.627553,-74.032040,11209,NY,Bay Ridge Preparatory School,v-1577758462
4,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4e8ef5bff7900fe594647fb4,7420 4th Ave,US,Brooklyn,United States,74th Street & 4th Avenue,205,"[7420 4th Ave (74th Street & 4th Avenue), Broo...","[{'label': 'display', 'lat': 40.63152460703191...",40.631525,-74.025188,11209,NY,Bay Ridge Preparatory School,v-1577758462


In [28]:
# keep only columns that include venue name, and anything that is associated with location
schools1_filtered_columns = ['name', 'categories'] + [col for col in schools1_df.columns if col.startswith('location.')] + ['id']
schools1_filtered = schools1_df.loc[:, schools1_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
schools1_filtered['categories'] = schools1_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
schools1_filtered.columns = [column.split('.')[-1] for column in schools1_filtered.columns]

schools1_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Vicky Simegiatos Dance School,Dance Studio,7110 3rd Ave,US,Brooklyn,United States,btwn 71st and 72nd St.,389,"[7110 3rd Ave (btwn 71st and 72nd St.), Brookl...","[{'label': 'display', 'lat': 40.63394249448927...",40.633942,-74.026807,11209,NY,4b92c39ef964a520991934e3
1,PS 102 - The Bay View School,General College & University,211 72nd St,US,Brooklyn,United States,Ridge Avenue,430,"[211 72nd St (Ridge Avenue), Brooklyn, NY 1120...","[{'label': 'display', 'lat': 40.63411946924494...",40.634119,-74.028804,11209,NY,4b2791e3f964a5207b8724e3
2,Lutheran elementary school,Elementary School,440 Ovington Ave,US,Brooklyn,United States,NaN,467,"[440 Ovington Ave, Brooklyn, NY 11209, United ...","[{'label': 'display', 'lat': 40.63303383059814...",40.633034,-74.022808,11209,NY,4c8a53481eafb1f7d9207935
3,Bay Ridge Preparatory School,Middle School,NaN,US,Brooklyn,United States,NaN,522,"[Brooklyn, NY 11209, United States]","[{'label': 'display', 'lat': 40.62755320651353...",40.627553,-74.032040,11209,NY,505c6db0e4b08887c5814fcb
4,Bay Ridge Preparatory School,High School,7420 4th Ave,US,Brooklyn,United States,74th Street & 4th Avenue,205,"[7420 4th Ave (74th Street & 4th Avenue), Broo...","[{'label': 'display', 'lat': 40.63152460703191...",40.631525,-74.025188,11209,NY,4e8ef5bff7900fe594647fb4


In [29]:
# Let's visualize the Schools1 that are nearby Location 1
schools1_filtered.name

0          Vicky Simegiatos Dance School
1           PS 102 - The Bay View School
2             Lutheran elementary school
3           Bay Ridge Preparatory School
4           Bay Ridge Preparatory School
5                    lama driving school
6    Voting at Fort Hamilton High School
7      Bay Ridge Holy Cross Greek School
8                  Bay Ridge Auto School
9                      Notre Dame School
Name: name, dtype: object

In [30]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=D0Y1ID3LKSEQNR01FLFZFRQR0AO45ISBTXZK3TMJKW3OXUUX&client_secret=ZDSX1VXNKJ1LOXXLPINFLJYOWMIWTN2AKERZQ50ZKAMR0TLP&ll=40.7583358,-73.969915265694&v=20191230&query=School&radius=500&limit=30'

In [31]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5e0aaeb4b4b684001b1e281b'},
 'response': {'venues': [{'id': '4bcd182e68f976b0c2e86383',
    'name': 'High School Of Art & Design',
    'location': {'address': '245 2nd Ave',
     'crossStreet': 'btwn E 56th St & E 57th St',
     'lat': 40.75897281748777,
     'lng': -73.96664813855,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75897281748777,
       'lng': -73.96664813855}],
     'distance': 284,
     'postalCode': '10022',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['245 2nd Ave (btwn E 56th St & E 57th St)',
      'New York, NY 10022',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d13d941735',
      'name': 'High School',
      'pluralName': 'High Schools',
      'shortName': 'High School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referra

In [32]:
# assign relevant part of JSON to schools2
schools2 = results2['response']['venues']

# tranform schools2 into a dataframe
schools2_df = json_normalize(schools2)
schools2_df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4bcd182e68f976b0c2e86383,245 2nd Ave,US,New York,United States,btwn E 56th St & E 57th St,284,"[245 2nd Ave (btwn E 56th St & E 57th St), New...","[{'label': 'display', 'lat': 40.75897281748777...",40.758973,-73.966648,10022,NY,High School Of Art & Design,v-1577758462,NaN
1,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",False,5416de6f498e9aee39129577,NaN,US,New York,United States,NaN,93,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.758582, 'lng':...",40.758582,-73.970978,NaN,NY,Cheung Kong Graduate School of Business,v-1577758462,NaN
2,"[{'id': '50328a8e91d4c4b30a586d6c', 'name': 'N...",False,4f5c0013e4b0c83983186872,244 E 52nd St,US,New York,United States,2nd Avenue,263,"[244 E 52nd St (2nd Avenue), New York, NY 1002...","[{'label': 'display', 'lat': 40.75637728334629...",40.756377,-73.968166,10022,NY,Turtle Bay Music School,v-1577758462,49218097
3,"[{'id': '4bf58dd8d48988d135941735', 'name': 'I...",False,4bfc1c83b5cd2d7f9f1a31fb,340 E 54th St,US,New York,United States,NaN,436,"[340 E 54th St, New York, NY 10022, United Sta...","[{'label': 'display', 'lat': 40.75680353586733...",40.756804,-73.965156,10022,NY,The Neighborhood Playhouse School Of The Theatre,v-1577758462,NaN
4,[],False,4b8d9f30f964a520cd0433e3,234 E 52nd St,US,New York,United States,NaN,250,"[234 E 52nd St, New York, NY 10022, United Sta...","[{'label': 'display', 'lat': 40.756321, 'lng':...",40.756321,-73.968599,10022,NY,Dave Frank School of Jazz,v-1577758462,NaN


In [33]:
# keep only columns that include venue name, and anything that is associated with location
schools2_filtered_columns = ['name', 'categories'] + [col for col in schools2_df.columns if col.startswith('location.')] + ['id']
schools2_filtered = schools2_df.loc[:, schools2_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
schools2_filtered['categories'] = schools2_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
schools2_filtered.columns = [column.split('.')[-1] for column in schools2_filtered.columns]

schools2_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,High School Of Art & Design,High School,245 2nd Ave,US,New York,United States,btwn E 56th St & E 57th St,284,"[245 2nd Ave (btwn E 56th St & E 57th St), New...","[{'label': 'display', 'lat': 40.75897281748777...",40.758973,-73.966648,10022,NY,4bcd182e68f976b0c2e86383
1,Cheung Kong Graduate School of Business,College Academic Building,NaN,US,New York,United States,NaN,93,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.758582, 'lng':...",40.758582,-73.970978,NaN,NY,5416de6f498e9aee39129577
2,Turtle Bay Music School,Non-Profit,244 E 52nd St,US,New York,United States,2nd Avenue,263,"[244 E 52nd St (2nd Avenue), New York, NY 1002...","[{'label': 'display', 'lat': 40.75637728334629...",40.756377,-73.968166,10022,NY,4f5c0013e4b0c83983186872
3,The Neighborhood Playhouse School Of The Theatre,Indie Theater,340 E 54th St,US,New York,United States,NaN,436,"[340 E 54th St, New York, NY 10022, United Sta...","[{'label': 'display', 'lat': 40.75680353586733...",40.756804,-73.965156,10022,NY,4bfc1c83b5cd2d7f9f1a31fb
4,Dave Frank School of Jazz,None,234 E 52nd St,US,New York,United States,NaN,250,"[234 E 52nd St, New York, NY 10022, United Sta...","[{'label': 'display', 'lat': 40.756321, 'lng':...",40.756321,-73.968599,10022,NY,4b8d9f30f964a520cd0433e3


In [34]:
# Let's visualize the Schools2 that are nearby Location 2
schools2_filtered.name

0                          High School Of Art & Design
1              Cheung Kong Graduate School of Business
2                              Turtle Bay Music School
3     The Neighborhood Playhouse School Of The Theatre
4                            Dave Frank School of Jazz
5       C.A.T.S. - Children's Athletic Training School
6                                   The Beekman School
7                         Martial Arts Training School
8                        Cats Dance Company and School
9                            St Albans Catholic School
10                             KERASTASE SCHOOL update
11         Montessori School of New York International
12        American Community School At Beirut -Lebanon
13              ESE - The European School of Economics
Name: name, dtype: object

# Visualization

In [35]:
schools_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the two locations and packaging centre

# add a red circle marker to represent Location1
folium.features.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Location1',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(schools_map)

# add a yellow circle marker to represent Location2
folium.features.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    color='yellow',
    popup='Location2',
    fill = True,
    fill_color = 'yellow',
    fill_opacity = 0.6
).add_to(schools_map)

# add the schools near location1 as blue circle markers
for lat, lng, label in zip(schools1_filtered.lat, schools1_filtered.lng,schools1_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(schools_map)

# add the schools near location2 as black circle markers
for lat, lng, label in zip(schools2_filtered.lat, schools2_filtered.lng,schools2_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(schools_map)

# display map
schools_map

In [36]:
# Both locations both seem to have a good number of schools nearby, we check for any trending venues to filter down
# define URL
url3 = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION)

# send GET request and get trending venues
results3 = requests.get(url3).json()
results3

{'meta': {'code': 200, 'requestId': '5e0aaeac1835dd001b5b033c'},
 'response': {'venues': []}}

In [37]:
# define URL: Location2
url4 = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION)

# send GET request and get trending venues
results4 = requests.get(url4).json()
results4

{'meta': {'code': 200, 'requestId': '5e0aaefb9da7ee001b9478ec'},
 'response': {'venues': [{'id': '49b7ed6df964a52030531fe3',
    'name': '타임스 스퀘어 (Times Square)',
    'location': {'address': 'Broadway & 7th Ave',
     'crossStreet': 'btwn 42nd & 47th St',
     'lat': 40.75808790683484,
     'lng': -73.98564005394907,
     'distance': 1326,
     'postalCode': '10036',
     'cc': 'US',
     'neighborhood': 'Theater District',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Broadway & 7th Ave (btwn 42nd & 47th St)',
      'New York, NY 10036',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d164941735',
      'name': 'Plaza',
      'pluralName': 'Plazas',
      'shortName': 'Plaza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '1717340',
     'url': 'https://www.seamless.com/menu/benihana-virtual-ti

In [38]:
# Check if any venues are trending at this time for Location2
if len(results4['response']['venues']) == 0:
    trending_venues_df2 = 'No trending venues are available at the moment!'
    
else:
    trending_venues2 = results4['response']['venues']
    trending_venues_df2 = json_normalize(trending_venues2)

    # filter columns
    columns_filtered2 = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df2 = trending_venues_df2.loc[:, columns_filtered2]

    # filter the category for each row
    trending_venues_df2['categories'] = trending_venues_df2.apply(get_category_type, axis=1)

In [39]:
# display trending venues for Location2
trending_venues_df2

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,타임스 스퀘어 (Times Square),Plaza,1326,New York,10036,NY,United States,40.758088,-73.985640
1,Broadhurst Theatre,Theater,1503,New York,10036,NY,United States,40.758132,-73.987744
2,OKLAHOMA!,Other Event,1310,New York,10019,NY,United States,40.762104,-73.984646
3,Cort Theatre,Theater,1148,New York,10036,NY,United States,40.759326,-73.983478
4,Rockefeller Center,Plaza,733,New York,10112,NY,United States,40.758668,-73.978604


In [40]:
# Visualization of trending venues and schools2 for Location 2
if len(results4['response']['venues']) == 0:
   location2_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    location2_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Location2


    # add Location2 as a yellow circle mark
    folium.features.CircleMarker(
        [latitude2, longitude2],
        radius=10,
        popup='Location2',
        fill=True,
        color='yellow',
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(location2_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df2['location.lat'], trending_venues_df2['location.lng'], trending_venues_df2['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='green',
            fill_color='green',
            fill_opacity=0.6
        ).add_to(location2_map)

    # add the schools near location2 as black circle markers
    for lat, lng, label in zip(schools2_filtered.lat, schools2_filtered.lng,schools2_filtered.categories):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='black',
            popup=label,
            fill = True,
            fill_color='black',
            fill_opacity=0.6
        ).add_to(location2_map)
        
# display map
location2_map

In [41]:
#Visualization for no trending venues and schools1 for Location 1
location1_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new warehouse and packaging centre

# add a red circle marker to represent Location1
folium.features.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Location1',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(location1_map)

# add the schools near location1 as blue circle markers
for lat, lng, label in zip(schools1_filtered.lat, schools1_filtered.lng,schools1_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(location1_map)
    
#display map
location1_map

# Thank you